In [2]:
'''import basic package'''
import pandas as pd
import numpy as np


'''import deep learning package'''
import keras as k
import tensorflow as tf


import requests
import json

from IPython.display import Image
from IPython.display import display
from IPython.display import HTML

import urllib.request

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage


Using TensorFlow backend.


In [3]:
rating_df = pd.read_csv('ml-latest-small/ratings.csv')
linkes_df = pd.read_csv('ml-latest-small/links.csv')

df_merged = pd.merge(rating_df,linkes_df,on=['movieId'])
print(df_merged.head())
print(df_merged.shape)

   userId  movieId  rating   timestamp  imdbId  tmdbId
0       1       31     2.5  1260759144  112792  9909.0
1       7       31     3.0   851868750  112792  9909.0
2      31       31     4.0  1273541953  112792  9909.0
3      32       31     4.0   834828440  112792  9909.0
4      36       31     3.0   847057202  112792  9909.0
(100004, 6)


In [ ]:
rating_matrix_dim = [len(df_merged.userId.unique()),max(df_merged.movieId)]

rating_matrix = np.zeros(rating_matrix_dim)


for row in df_merged.itertuples():
    try:
        rating_matrix[row[1]-1, row[2]-1] = row[3]
    except:
        continue

rating_matrix = rating_matrix[:,:9000]

print(rating_matrix.shape)

In [ ]:
np.save('rating_matrix',rating_matrix)

In [ ]:
'''built a map between movieId and tmdbID'''
movieId_vs_tmdbId = {}

for row in df_merged[['movieId','tmdbId']].itertuples():    
    try:
        movieId_vs_tmdbId[str(int(row[1]-1))] = int(row[2])
    except:
        continue

np.save('movieId_vs_tmdbId',movieId_vs_tmdbId)

In [ ]:
'''this cell just run one time to download all posters'''
# poster_path = "/Users/zishuoli/Downloads/bigdata/final/posters/"

movie_list = []
# item in movie list is tmdb id
for i in movieId_vs_tmdbId.values():
    try:
        poster_url = get_poster(i)
        # download poster jpg to poster_path
        urllib.request.urlretrieve(poster_url, poster_path + str(int(i)) + ".jpg")
        movie_list.append(i)
    except:
        print('tmdb movie id :',str(i),'is not available')
        continue


In [ ]:

'''transfer learning'''
img = []

poster_path = "/Users/zishuoli/Downloads/bigdata/final/posters/"

for i in movie_list:
    tmp = kimage.load_img(poster_path + str(i) + ".jpg", target_size=(224, 224))
    tmp = kimage.img_to_array(tmp)
    tmp = np.expand_dims(tmp, axis=0)
    tmp = preprocess_input(tmp)   
    
    img.append(tmp)
    
# image_top=False removes final connected layers   
model = VGG16(include_top=False, weights='imagenet')

In [ ]:
movie_num = len(movie_list)

feature_matrix = np.zeros([movie_num,25088])

for i in range(movie_num):
    tmp = model.predict(img[i]).ravel()
    feature_matrix[i,:] = tmp

In [ ]:
np.save('feature_matrix',feature_matrix)
np.save('movie_list',movie_list)